<div style="width:100%; background-color: #000041"><a target="_blank" href="http://university.yugabyte.com"><img src="assets/YBU_Logo.png" /></a></div><br>

> **Migrate and Modernize with YugabyteDB Voyager**
>
> Enroll for free at [Yugabyte University](https://university.yugabyte.com/courses/YugabyteDB-Voyager).
>

<br>
This notebook file is:

`04_Migration_Workflow_Migrate_Schema_2.ipynb`

## 🛠️ Requirements
Here are the requirements for this notebook:
- ✅ Create the notebook variables in `01_Lab_Setup.ipynb`, which you previously did
- ✅ Create the `db_ybu` database, which you previously did
- ☑️ A Python 3.12 or higher kernel for the notebook, *which you need to select right now!!!*
- ☑️ Import the notebook variables, *which you must do next*
- ☑️ Migrate schema workflow (2), *which you must do next*


### Select your notebook kernel
- In the Notebook toolbar, click **Select Kernel**.
<br>
<img width=50% src="assets/01_01_Select_Kernel_Toolbar.png" />

- Next, in the dropdown, select **Python 3.12** or higher.
<br>
<img width=50% src="assets/01_02_Select_Kernel_Dropdown.png" />

That's it!

## ⛑️ Getting help
The best way to get help us from the Yugabyte University team. You can post your question in YugabyteDB Community Slack in the #training or #yb-university channels. Join our [community](https://communityinviter.com/apps/yugabyte-db/register).

## 👣 Set up steps for this notebook
Here are the steps to setup this lab:

- Import the notebook variables

### 👇 Import the notebook variables

> 👉 IMPORTANT! 👈
> 
> Do **NOT** skip running the following cell. 
> 

The following Python cell reads the stored variables created in the `01_Lab_Setup.ipynb` notebook. To run the script, select Execute Cell (Play Arrow) in the left gutter of the cell. 

👇 👇 👇 

In [ ]:
# Use %store -r to read 01_Lab_Setup variables
%store -r 
%config SqlMagic.autopandas=False
# %config SqlMagic.named_parameters=True
%config SqlMagic.displaylimit=30
%config SqlMagic.displaycon=False

**Update the notebook styling**

In [ ]:
from IPython.core.display import HTML
def css_styling():
    styles = open("./styles/custom.css", "r").read()
    return HTML(styles)
css_styling()

---
# Workflow: Migrate schema (2)


<div style="width:100%; background-color: #000041"><img src="assets/MVOY100_03_2-Migrate_Schema_2.png" /></div>

---
# Import the modified schema

 `yb-voyager` imports the source database into the `public` schema of the target database. This is applicable only for MySQL and Oracle databases. 
 
 By specifying `--target-db-schema` argument during import, you can instruct `yb-voyager` to create a non-public schema and use it for the import.



In [ ]:
%%bash

yb-voyager import schema --help

Import the modified schema to the target YugabyteDB database using the `yb-voyager import schema` command.

In [ ]:
%%bash -s "$NB_DB_NAME"  "$NB_DB_PORT" "$NB_HOST_IPv4_01" "$NB_NOTEBOOK_VOYAGER_TMP"
DB_NAME=${1}
DB_PORT=${2}
DB_HOST=${3}
NOTEBOOK_VOYAGER_TMP=${4}

yb-voyager import schema  -y  \
  --start-clean true \
  --export-dir $NOTEBOOK_VOYAGER_TMP/export-dir \
  --target-db-host ${DB_HOST} \
  --target-db-port  ${DB_PORT} \
  --target-db-name  ${DB_NAME} \
  --target-db-user "ybvoyager" \
  --target-db-password "Password" \
  --target-db-schema "sakila"


That raised an error: 

`ERROR: in an aggregate with DISTINCT, ORDER BY expressions must appear in argument list (SQLSTATE 42P10)`

There may be more than one!

---
## Resolve the schema import issues

Oh no! Errors!

Grep the log files to get a better understanding of the related errors. Start with the first one that has to do with a table. Typically resolving errors with the DDL of a table will help solve other errors with views and other dependent objects.

In [ ]:
%%bash -s "$NB_NOTEBOOK_VOYAGER_TMP"
NOTEBOOK_VOYAGER_TMP=${1}
IMPORT_SCHEMA_LOG=yb-voyager-import-schema.log
 
tail -n 2000 $NOTEBOOK_VOYAGER_TMP/export-dir/logs/$IMPORT_SCHEMA_LOG| grep 'SQLSTATE'

The logs reveal two main issues:
-  `DDL Execution Failed`
-  `ERROR: in an aggregate with DISTINCT, ORDER BY expressions must appear in argument list (SQLSTATE 42P10)`.  

The first issue `DDL Execution Failed` issue is with the `CREATE TABLE address`. There is an error code for the failed execution:
`ERROR: type "geometry" does not exist (SQLSTATE 42704)`.

Begin by resolving the issue this `CREATE TABLE address` statement.

```
ERROR importData.go:942 DDL Execution Failed for "CREATE TABLE address (\n\taddress_id serial,\n\taddress varchar(50) NOT NULL,\n\taddress2 varchar(50),\n\tdistrict varchar(20) NOT NULL,\n\tcity_id integer NOT NULL,\n\tpostal_code varchar(10),\n\tphone varchar(20) NOT NULL,\n\tlocation GEOMETRY NOT NULL,\n\tlast_update timestamp NOT NULL DEFAULT CURRENT_TIMESTAMP,\n\tPRIMARY KEY (address_id)\n) ;": ERROR: type "geometry" does not exist (SQLSTATE 42704)
```

View the table definition from the source database as well as row.

In [ ]:
%%bash -s "$NB_MYSQL_DB_NAME"  # create database
DB_NAME=${1}

mysql -h 0 -u root ${DB_NAME} -e "SHOW CREATE TABLE address; "  
mysql -h 0 -u root ${DB_NAME} -e "select location from address where location is not null limit 1; "  

The GEOMETRY datatype for  MYSQL 8.0+ an be in several formats:
- Well-Known Text (WKT) Format
- Well-Known Binary (WKB) Format
- Internal Geometry Storage Format

> Learn more
> 
> For more details, visit:
> [https://docs.yugabyte.com/preview/api/ysql/datatypes/type_binary/](https://docs.yugabyte.com/preview/api/ysql/datatypes/type_binary/)

Modify the DDL of the `address` table to use a different datatype in the export schema file. Change the data type to a binary data type, `BYTEA`, which is variable length binary string. 

In [ ]:
%%bash -s "$NB_NOTEBOOK_VOYAGER_TMP"
NOTEBOOK_VOYAGER_TMP=${1}

SEARCH="GEOMETRY"
REPLACE="BYTEA"
FILE_PATH=$NOTEBOOK_VOYAGER_TMP/export-dir/schema/tables/table.sql

sed -i "s/$SEARCH/$REPLACE/" $FILE_PATH

Confirm the change to the file.

In [ ]:
%%bash -s "$NB_NOTEBOOK_VOYAGER_TMP"
NOTEBOOK_VOYAGER_TMP=${1}

FILE_PATH=$NOTEBOOK_VOYAGER_TMP/export-dir/schema/tables/table.sql

cat $FILE_PATH  | grep -B 10 'BYTEA'

The next issues is with the `actor_info` view. The DDL failed due because of issue with the keyword `distinct`:
-  `ERROR: in an aggregate with DISTINCT, ORDER BY expressions must appear in argument list (SQLSTATE 42P10)`.  

Continue by resolving this issue.

In [ ]:
%%bash -s "$NB_NOTEBOOK_VOYAGER_TMP"
NOTEBOOK_VOYAGER_TMP=${1}

FILE_PATH=$NOTEBOOK_VOYAGER_TMP/export-dir/schema/views/view.sql

cat $FILE_PATH  | grep -A 1 'CREATE OR REPLACE VIEW actor_info'

The challenge here has to do with the functions, `array_to_string() and array_agg()` . 

These are being used to replace the `group_concat()` built-in function of MySQL. 

The best solution, which is long, is to comment out the view, and after successfully importing the schema and data, do the following: 
- create the `_group_concat` function 
- create the `group_concat` aggregate function 
- create the `actor_info` view that uses the `group_concat` function 

The solution looks something like the following, but **don't execute this cell**, just review the code:

In [ ]:
/* Do NOT run this cell, it's for illustration only */
CREATE FUNCTION _group_concat(text, text) RETURNS text
    AS $_$
SELECT CASE
  WHEN $2 IS NULL THEN $1
  WHEN $1 IS NULL THEN $2
  ELSE $1 || ', ' || $2
END
$_$
LANGUAGE sql IMMUTABLE;


ALTER FUNCTION sakila._group_concat(text, text) OWNER TO yugabyte;


CREATE AGGREGATE group_concat(text) (
    SFUNC = _group_concat,
    STYPE = text
);


ALTER AGGREGATE sakila.group_concat(text) OWNER TO yugabyte;

DROP VIEW  actor_info;
CREATE VIEW actor_info AS
    SELECT a.actor_id, 
    a.first_name, 
    a.last_name, 
    group_concat(DISTINCT (((c.name)::text || ': '::text) || (SELECT group_concat((f.title)::text) AS group_concat FROM ((film f JOIN film_category fc ON ((f.film_id = fc.film_id))) JOIN film_actor fa ON ((f.film_id = fa.film_id))) WHERE ((fc.category_id = c.category_id) AND (fa.actor_id = a.actor_id)) GROUP BY fa.actor_id))) AS film_info 
    FROM (((actor a LEFT JOIN film_actor fa ON ((a.actor_id = fa.actor_id))) LEFT JOIN film_category fc ON ((fa.film_id = fc.film_id))) LEFT JOIN category c ON ((fc.category_id = c.category_id))) 
    GROUP BY a.actor_id, a.first_name, a.last_name;

But for now, simply remove the distinct clause from the view.

In [ ]:
%%bash -s "$NB_NOTEBOOK_VOYAGER_TMP"
NOTEBOOK_VOYAGER_TMP=${1}

SEARCH="distinct"
REPLACE=""
FILE_PATH=$NOTEBOOK_VOYAGER_TMP/export-dir/schema/views/view.sql

sed -i "s/$SEARCH/$REPLACE/" $FILE_PATH

---
## Rinse & Repeat (1): Import the modified schema and resolve DDL errors

In [ ]:
%%bash -s "$NB_DB_NAME"  "$NB_DB_PORT" "$NB_HOST_IPv4_01" "$NB_NOTEBOOK_VOYAGER_TMP"
DB_NAME=${1}
DB_PORT=${2}
DB_HOST=${3}
NOTEBOOK_VOYAGER_TMP=${4}
IMPORT_SCHEMA_LOG=yb-voyager-import-schema.log

# delete the log so that we can get a fresh view
rm -rf $NOTEBOOK_VOYAGER_TMP/export-dir/logs/$IMPORT_SCHEMA_LOG

yb-voyager import schema -y  \
  --start-clean true \
  --export-dir $NOTEBOOK_VOYAGER_TMP/export-dir \
  --target-db-host ${DB_HOST} \
  --target-db-port  ${DB_PORT} \
  --target-db-name  ${DB_NAME} \
  --target-db-user "ybvoyager" \
  --target-db-password "Password" \
  --target-db-schema "sakila" 


Ah, more import schema errors! Take a look at the `yb-voyager-import-schema.log` file.

In [ ]:
%%bash -s "$NB_NOTEBOOK_VOYAGER_TMP"
NOTEBOOK_VOYAGER_TMP=${1}
IMPORT_SCHEMA_LOG=yb-voyager-import-schema.log
 
tail -n 2000 $NOTEBOOK_VOYAGER_TMP/export-dir/logs/$IMPORT_SCHEMA_LOG| grep 'SQLSTATE'

Resolve another error. In this case, look at the error for the view, `customer_list`.

```
DDL Execution Failed for "CREATE OR REPLACE VIEW customer_list AS select cu.customer_id AS ID,concat(cu.first_name,' ',cu.last_name) AS name,a.address AS address,a.postal_code AS zip code,a.phone AS phone,city.city AS city,country.country AS country,CASE WHEN cu.active THEN 'active' ELSE '' END  AS notes,cu.store_id AS SID FROM (((customer cu join address a on (cu.address_id = a.address_id)) join city on (a.city_id = city.city_id)) join country on (city.country_id = country.country_id));": ERROR: syntax error at or near "code" (SQLSTATE 42601)
```


Use sed or the open editor to make your changes.

In [ ]:
%%bash -s "$NB_NOTEBOOK_VOYAGER_TMP"
NOTEBOOK_VOYAGER_TMP=${1}

SEARCH="zip code"
REPLACE="zip_code"
FILE_PATH=$NOTEBOOK_VOYAGER_TMP/export-dir/schema/views/view.sql

sed -i "s/$SEARCH/$REPLACE/" $FILE_PATH
gp open $FILE_PATH

Verify the change.

In [ ]:
%%bash -s "$NB_NOTEBOOK_VOYAGER_TMP"
NOTEBOOK_VOYAGER_TMP=${1}

FILE_PATH=$NOTEBOOK_VOYAGER_TMP/export-dir/schema/views/view.sql

cat $FILE_PATH  | grep 'zip_code'

The operation resolved two errors with the space in the `zip code` column name, for both the  `customer_list` and `staff_list` views.

---
## Rinse & Repeat (2): Import the modified schema and resolve DDL errors

In [ ]:
%%bash -s "$NB_DB_NAME"  "$NB_DB_PORT" "$NB_HOST_IPv4_01" "$NB_NOTEBOOK_VOYAGER_TMP"
DB_NAME=${1}
DB_PORT=${2}
DB_HOST=${3}
NOTEBOOK_VOYAGER_TMP=${4}
IMPORT_SCHEMA_LOG=yb-voyager-import-schema.log

# delete the log so that we can get a fresh view
rm -rf $NOTEBOOK_VOYAGER_TMP/export-dir/logs/$IMPORT_SCHEMA_LOG

yb-voyager import schema  -y  \
  --start-clean true \
  --export-dir $NOTEBOOK_VOYAGER_TMP/export-dir \
  --target-db-host ${DB_HOST} \
  --target-db-port  ${DB_PORT} \
  --target-db-name  ${DB_NAME} \
  --target-db-user "ybvoyager" \
  --target-db-password "Password" \
  --target-db-schema "sakila" 


Another error!

`ERROR: argument of CASE/WHEN must be type boolean, not type smallint (SQLSTATE 42804)`

Look at the log for more details.

In [ ]:
%%bash -s "$NB_NOTEBOOK_VOYAGER_TMP"
NOTEBOOK_VOYAGER_TMP=${1}
IMPORT_SCHEMA_LOG=yb-voyager-import-schema.log
 
tail -n 2000 $NOTEBOOK_VOYAGER_TMP/export-dir/logs/$IMPORT_SCHEMA_LOG | grep 'SQLSTATE'

```
 Execution Failed for "CREATE OR REPLACE VIEW customer_list ....
 CASE WHEN cu.active THEN 'active' ELSE '' END 
```

The `cu` alias is the  `customer` table, and the issue is with the `active` column.

In [ ]:
%%bash -s "$NB_NOTEBOOK_VOYAGER_TMP"
NOTEBOOK_VOYAGER_TMP=${1}

FILE_PATH=$NOTEBOOK_VOYAGER_TMP/export-dir/schema/tables/table.sql

cat $FILE_PATH  | grep -A 11 'CREATE TABLE customer'

The `active` column uses `smallint`. 

Update the `table.sql` file using sed or the editor. 

Change the `smallint` to a `boolean`.

In [ ]:
%%bash -s "$NB_NOTEBOOK_VOYAGER_TMP"
NOTEBOOK_VOYAGER_TMP=${1}

SEARCH="active smallint NOT NULL DEFAULT 1"
REPLACE="active boolean NOT NULL DEFAULT TRUE"
FILE_PATH=$NOTEBOOK_VOYAGER_TMP/export-dir/schema/tables/table.sql

sed -i "s/$SEARCH/$REPLACE/" $FILE_PATH
gp open $FILE_PATH


Verify the changes

In [ ]:
%%bash -s "$NB_NOTEBOOK_VOYAGER_TMP"
NOTEBOOK_VOYAGER_TMP=${1}

FILE_PATH=$NOTEBOOK_VOYAGER_TMP/export-dir/schema/tables/table.sql

cat $FILE_PATH  | grep -A 11 'CREATE TABLE customer'

---
## Rinse & Repeat (3): Import the modified schema and resolve DDL errors

Rerun `import schema` again.

In [ ]:
%%bash -s "$NB_DB_NAME"  "$NB_DB_PORT" "$NB_HOST_IPv4_01" "$NB_NOTEBOOK_VOYAGER_TMP"
DB_NAME=${1}
DB_PORT=${2}
DB_HOST=${3}
NOTEBOOK_VOYAGER_TMP=${4}
IMPORT_SCHEMA_LOG=yb-voyager-import-schema.log

# delete the log so that we can get a fresh view
rm -rf $NOTEBOOK_VOYAGER_TMP/export-dir/logs/$IMPORT_SCHEMA_LOG

yb-voyager import schema  -y  \
  --start-clean true \
  --export-dir $NOTEBOOK_VOYAGER_TMP/export-dir \
  --target-db-host ${DB_HOST} \
  --target-db-port  ${DB_PORT} \
  --target-db-name  ${DB_NAME} \
  --target-db-user "ybvoyager" \
  --target-db-password "Password" \
  --target-db-schema "sakila" 

Oops! Another error:

`error: ERROR: column "c.city" must appear in the GROUP BY clause or be used in an aggregate function (SQLSTATE 42803)`

This error has to do the view, `sales_by_store`:

```
DDL Execution Failed for "CREATE OR REPLACE VIEW sales_by_store AS select concat(c.city,',',cy.country) AS store,concat(m.first_name,' ',m.last_name) AS manager,sum(p.amount) AS total_sales FROM (((((((payment p join rental r on (p.rental_id = r.rental_id)) join inventory i on (r.inventory_id = i.inventory_id)) join store s on (i.store_id = s.store_id)) join address a on (s.address_id = a.address_id)) join city c on (a.city_id = c.city_id)) join country cy on (c.country_id = cy.country_id)) join staff m on (s.manager_staff_id = m.staff_id)) group by s.store_id order by cy.country,c.city;": ERROR: column "c.city" must appear in the GROUP BY clause or be used in an aggregate function (SQLSTATE 42803)
```

PostgreSQL, like most sensible DBMSs, does `group by` properly, insisting that everything that you select must be either "grouped by" or be aggregated through a function, like sum(). 

Unfortunately, MySQL is not as strict.

To resolve this, you will need to replace the view in the `view.sql` file.

Copy the following SQL.

In [ ]:
/* Do not run this cell. 
   The view.sql file tab is open.
   Copy and paste the following into the view.sql file 
   Comment out  (--) the original sales_by_store view */
   
CREATE OR REPLACE VIEW sales_by_store AS 
select 
    concat(c.city,',',cy.country) AS store,
    concat(m.first_name,' ',m.last_name) AS manager,
    sum(p.amount) AS total_sales 
FROM (((((((payment p join rental r on (p.rental_id = r.rental_id)) join inventory i on (r.inventory_id = i.inventory_id)) join store s on (i.store_id = s.store_id)) join address a on (s.address_id = a.address_id)) join city c on (a.city_id = c.city_id)) join country cy on (c.country_id = cy.country_id)) join staff m on (s.manager_staff_id = m.staff_id))
group by c.city, cy.country, m.first_name, m.last_name
order by cy.country, c.city;

And then open the `view.sql` file, comment out the old view, and then paste in the new one.

In [ ]:
%%bash -s "$NB_NOTEBOOK_VOYAGER_TMP"
NOTEBOOK_VOYAGER_TMP=${1}

FILE_PATH=$NOTEBOOK_VOYAGER_TMP/export-dir/schema/views/view.sql

gp open $FILE_PATH

Remember to comment out line `11`, the original `sales_by_store` view.

Save your changes to the `view.sql` file.

---
## Rinse & Repeat (4): Import the modified schema and resolve DDL errors

Rerun `import schema` again.

In [ ]:
%%bash -s "$NB_DB_NAME"  "$NB_DB_PORT" "$NB_HOST_IPv4_01" "$NB_NOTEBOOK_VOYAGER_TMP"
DB_NAME=${1}
DB_PORT=${2}
DB_HOST=${3}
NOTEBOOK_VOYAGER_TMP=${4}
IMPORT_SCHEMA_LOG=yb-voyager-import-schema.log

# delete the log so that we can get a fresh view
rm -rf $NOTEBOOK_VOYAGER_TMP/export-dir/logs/$IMPORT_SCHEMA_LOG

yb-voyager import schema  -y  \
  --start-clean true \
  --export-dir $NOTEBOOK_VOYAGER_TMP/export-dir \
  --target-db-host ${DB_HOST} \
  --target-db-port  ${DB_PORT} \
  --target-db-name  ${DB_NAME} \
  --target-db-user "ybvoyager" \
  --target-db-password "Password" \
  --target-db-schema "sakila" 

Great! That worked!

No errors! 

Nicely done 🙌 

> Note: `failed.sql`
>
> Depending on the type of error or erros, the `import schema` command may fail silently. 
> 
> Check to see if there is a `failed.sql` file in the `schema` directory.
> 

---
## Sanity check: Verify the imported schema in YugabyteDB

First, list the schemas using the `\dn` meta command.

In [ ]:
%%bash -s "$NB_YB_PATH_BIN" "$NB_DB_NAME"  # create database
YB_PATH_BIN=${1}
DB_NAME=${2}

cd $YB_PATH_BIN

# meta commands

./ysqlsh -d ${DB_NAME} -U ybvoyager -c "\dn"

> Note: `ybvoyager_metadata`
>
> Voyager creates it's own database from keeping track of the migration in the target database. The database name is `ybvoyager_metadata`.
> 

Next, list the tables using the `\dt` meta command.

In [ ]:
%%bash -s "$NB_YB_PATH_BIN" "$NB_DB_NAME"  # create database
YB_PATH_BIN=${1}
DB_NAME=${2}

cd $YB_PATH_BIN

# meta commands

./ysqlsh -d ${DB_NAME} -U ybvoyager -c "\dt sakila."


Verify the views using the `\dv` meta command.

In [ ]:
%%bash -s "$NB_YB_PATH_BIN" "$NB_DB_NAME"  # create database
YB_PATH_BIN=${1}
DB_NAME=${2}

cd $YB_PATH_BIN

./ysqlsh -d ${DB_NAME} -U ybvoyager -c "\dv sakila."

Verify the index creation with a PostgreSQL query. This may not be a complete list as additional indexes may be created after importing data. Typically, these indexes are unique indexes for primary keys.


In [ ]:
%%bash -s "$NB_YB_PATH_BIN" "$NB_DB_NAME"  # create database
YB_PATH_BIN=${1}
DB_NAME=${2}
SQL_PG_INDEXES="select tablename, indexname, indexdef from pg_indexes where schemaname = 'sakila' order by tablename, indexname;"
cd $YB_PATH_BIN

# query

./ysqlsh -d ${DB_NAME} -U ybvoyager -c "${SQL_PG_INDEXES}"

Verify the trigger creation with a PostgreSQL query. This may not be a complete list as additional triggers may be created after importing data.

In [ ]:
%%bash -s "$NB_YB_PATH_BIN" "$NB_DB_NAME"  # create database
YB_PATH_BIN=${1}
DB_NAME=${2}

SQL_PG_TRIGGERS="select  event_object_table AS table_name, trigger_name from information_schema.triggers where trigger_schema = 'sakila' group by table_name, trigger_name order by table_name, trigger_name;"


cd $YB_PATH_BIN
./ysqlsh -d ${DB_NAME} -U ybvoyager -c "${SQL_PG_TRIGGERS}"

---

## 😊 Next up!
Continue your learning by opening the next notebook, `05_Migration_Workflow_Migrate_Data.ipynb`. 

Or, if using GitPod, run the following cell:

In [ ]:
%%bash
gp open '05_Migration_Workflow_Migrate_Data.ipynb'